In [0]:
# 02_silver_transform
from pyspark.sql.functions import col, when, trim

# Use the DB database we created earlier
spark.sql("USE churn_mlo_mdb")

# Source bronze table created earlier
bronze = "churn_mlo_mdb.bronze_churn"
print("Reading bronze table:", bronze)
df = spark.table(bronze)

# Quick schema / sanity
print("Bronze schema:")
df.printSchema()

# 1) Trim whitespace for string columns (safe)
string_cols = [f.name for f in df.schema.fields if f.dataType.simpleString() == "string"]
for c in string_cols:
    df = df.withColumn(c, trim(col(c)))

# 2) Fix TotalCharges: empty strings -> null, cast to double
df = df.withColumn("TotalCharges", when(col("TotalCharges") == "" , None).otherwise(col("TotalCharges").cast("double")))

# 3) Drop rows with missing essential keys
df = df.dropna(subset=["customerID", "Churn"])

# 4) Create numeric churn label
df = df.withColumn("churn_label", when(col("Churn") == "Yes", 1).otherwise(0))

# 5) Remove duplicates (if any)
df = df.dropDuplicates(["customerID"])

# 6) Select a stable set of columns for silver (you can expand later)
keep_cols = ["customerID","gender","SeniorCitizen","Partner","Dependents","tenure",
             "PhoneService","InternetService","MonthlyCharges","TotalCharges",
             "Contract","PaymentMethod","churn_label","ingestion_time"]
keep_cols = [c for c in keep_cols if c in df.columns]

df_silver = df.select(*keep_cols)

# 7) Write Silver table as Delta
silver_table = "churn_mlo_mdb.silver_churn"
spark.sql(f"CREATE DATABASE IF NOT EXISTS churn_mlo_mdb")
spark.sql(f"DROP TABLE IF EXISTS {silver_table}")

df_silver.write.format("delta").mode("overwrite").saveAsTable(silver_table)

print("Silver table created:", silver_table)
display(spark.table(silver_table).limit(10))
